In [3]:
# from pymongo import MongoClient
# client = MongoClient("")
from pymongo import MongoClient
import pandas as pd
import time

In [125]:
client = MongoClient("localhost:27017")

In [126]:
db = client["Ironhack"]

In [127]:
#geting the dataset
c=db.get_collection("Companies")

In [128]:
# filter for the design companies
#only 4
filter_design = {"category_code":"design"}
projection_design = {"_id":0, "name":1, "offices.city":1}
list(c.find(filter_design, projection_design))

[{'name': '99designs',
  'offices': [{'city': 'San Francisco'},
   {'city': 'Collingwood'},
   {'city': 'Berlin'}]},
 {'name': 'Graticle', 'offices': [{'city': 'Ellensburg'}]},
 {'name': 'Kickstarter', 'offices': [{'city': 'Brooklyn'}]},
 {'name': 'Moonfruit', 'offices': [{'city': 'London'}]}]

In [129]:
#checking for the industry to define tech industry
list(c.distinct("category_code"))

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

In [130]:
#list with tech companies
tech = ['biotech','cleantech','ecommerce','games_video','nanotech','software','web']

In [131]:
#check list of latest founded year
list(c.distinct("founded_year"))[-4:]

[2010, 2011, 2012, 2013]

In [132]:
c.find_one()["funding_rounds"][0]['raised_amount']

5250000

In [133]:
# filter tech companies for startup criteria
filter_tech = {"category_code":{"$in":tech}}
filter_year = {"founded_year":{"$gte":2000}}
filter_funding = {"funding_rounds.raised_amount":{"$gte":1000000}}
filter_employees={"number_of_employees":{"$lte":100}}

In [134]:
#list with all filters
all_filters=[ filter_tech, filter_year, filter_funding, filter_employees ]

In [139]:
#
projection_3 = {"_id":0,"offices.city":1}
x=list(c.find({"$and": all_filters}, projection_3))
x

[{'offices': [{'city': 'Seattle'}, {'city': 'New York'}]},
 {'offices': [{'city': 'West Hollywood'}]},
 {'offices': [{'city': 'New York'}]},
 {'offices': [{'city': 'Sunnyvale'}]},
 {'offices': [{'city': 'Culver City'}]},
 {'offices': [{'city': 'San Francisco'}]},
 {'offices': [{'city': 'New York'}]},
 {'offices': [{'city': 'Pleasanton'}]},
 {'offices': [{'city': 'Luxembourg City'},
   {'city': 'Palo Alto'},
   {'city': 'London'}]},
 {'offices': [{'city': 'San Bruno'}]},
 {'offices': [{'city': 'NYC'}]},
 {'offices': [{'city': 'New York'}]},
 {'offices': [{'city': 'Encino'}]},
 {'offices': [{'city': 'San Francisco'}]},
 {'offices': [{'city': 'Amsterdam'}]},
 {'offices': [{'city': 'Redwood City'}]},
 {'offices': [{'city': 'San Francisco'}]},
 {'offices': [{'city': 'Paris'}, {'city': 'San Francisco'}]},
 {'offices': [{'city': 'San Francisco'}]},
 {'offices': [{'city': 'Seattle'}]},
 {'offices': [{'city': 'Palo Alto'}]},
 {'offices': [{'city': 'Seattle'}]},
 {'offices': [{'city': 'San Franc

In [138]:
from collections import Counter


In [143]:
Counter( [i['city'] for i in x["offices"]])

TypeError: list indices must be integers or slices, not str

In [146]:
cities=[i['offices'] for i in x]
print(cities) 

[[{'city': 'Seattle'}, {'city': 'New York'}], [{'city': 'West Hollywood'}], [{'city': 'New York'}], [{'city': 'Sunnyvale'}], [{'city': 'Culver City'}], [{'city': 'San Francisco'}], [{'city': 'New York'}], [{'city': 'Pleasanton'}], [{'city': 'Luxembourg City'}, {'city': 'Palo Alto'}, {'city': 'London'}], [{'city': 'San Bruno'}], [{'city': 'NYC'}], [{'city': 'New York'}], [{'city': 'Encino'}], [{'city': 'San Francisco'}], [{'city': 'Amsterdam'}], [{'city': 'Redwood City'}], [{'city': 'San Francisco'}], [{'city': 'Paris'}, {'city': 'San Francisco'}], [{'city': 'San Francisco'}], [{'city': 'Seattle'}], [{'city': 'Palo Alto'}], [{'city': 'Seattle'}], [{'city': 'San Francisco'}], [{'city': 'Charlottesville'}], [{'city': 'San Mateo'}], [{'city': 'San Diego'}, {'city': 'Los Angeles'}], [{'city': 'San Francisco'}], [{'city': 'San Francisco'}], [{'city': 'San Francisco'}], [{'city': 'Montecito'}], [{'city': 'San Mateo'}], [{'city': 'San Francisco'}, {'city': 'Huntsville'}], [{'city': 'San Mateo'

In [150]:
cities[0][0]

{'city': 'Seattle'}

In [148]:
cities_2=[i['city'][0] for i in cities]


TypeError: list indices must be integers or slices, not str